### Import data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib as map

In [ ]:
from pathlib import Path

# go up one folder from notebooks → project root
ROOT = Path.cwd().parent
RAW = ROOT / "data" / "raw"

import pandas as pd
df = pd.read_csv(RAW / "wellness.csv")
df.head(2)



,Timestamp,Today's Date,How did you feel today?,Whoop recovery score,Resting RHR,Calories,Whoop sleep score %,Whoop sleep hours,Trained?,Any notes?
0,9/26/2025 11:10:28,9/26/2025,2,Green,62,Deficit,85,6:15,Yes,Felt somewhat sick all day. Very drained. Low ...
